In [ ]:
import pandas as pd
import xlsxwriter 
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import gridspec
import copy

file_path = 'Input/premise_scenario_report.xlsx'  # Replace with the path to your Excel file
sheet_name = 'Electricity - generation'
scenario = 'REMIND - SSP2-BASE'
year = 2040

### Extract Data

In premise 1.6.7 these are the electricity generation technologies:

In [ ]:
techs_image = ["Biomass CHP", "Biomass CHP CCS", "Biomass ST", "Biomass IGCC CCS", "Biomass IGCC", "Coal PC", "Coal IGCC", "Coal IGCC CCS", "Coal CHP", "Coal CHP CCS", "Gas OC", "Gas CC", "Gas CHP", "Gas CHP CCS", "Gas CC CCS", "Geothermal", "Hydro", "Nuclear", "Oil ST", "Oil CC", "Oil CC CCS", "Oil CHP", "Oil CHP CCS", "Solar CSP", "Solar PV Centralized", "Solar PV Residential", "Wind Onshore", "Wind Offshore"]
techs_remind = ["Biomass CHP", "Biomass IGCC CCS", "Biomass IGCC", "Coal PC", "Coal IGCC", "Coal PC CCS", "Coal IGCC CCS", "Coal CHP", "Gas OC", "Gas CC", "Gas CHP", "Gas CC CCS", "Geothermal", "Hydro", "Nuclear", "Oil ST", "Solar CSP", "Solar PV Centralized", "Wind Onshore", "Wind Offshore"]

These are the years:

In [ ]:
years = [2005,2010,2015,2020,2025,2030,2035,2040,2045,2050, 2055, 2060,2070,2080,2090,2100]

These are the regions:

In [ ]:
regions_remind = ['CAZ', 'CHA', 'EUR', 'IND', 'JPN', 'LAM', 'MEA', 'NEU', 'OAS', 'REF', 'SSA', 'USA']
regions_image = ['BRA','CAN','CEU','CHN','EAF','INDIA','INDO','JAP','KOR','ME','MEX','NAF','OCE','RCAM','RSAF','RSAM','RSAS','RUS','SAF','SEAS','STAN','TUR','UKR','USA','WAF','WEU','World']

These are the scenarios:

In [ ]:
scenarios = [
    "IMAGE - SSP1-BASE",
    "IMAGE - SSP2-BASE",
    "IMAGE - SSP2-RCP26",
    "IMAGE - SSP2-RCP19",
    "REMIND - SSP1-BASE",
    "REMIND - SSP1-NDC",
    "REMIND - SSP1-NPI",
    "REMIND - SSP1-PKBUDG1150",
    "REMIND - SSP1-PKBUDG500",
    "REMIND - SSP2-BASE",
    "REMIND - SSP2-NDC",
    "REMIND - SSP2-NPI",
    "REMIND - SSP2-PKBUDG1150",
    "REMIND - SSP2-PKBUDG500",
    "REMIND - SSP5-BASE",
    "REMIND - SSP5-NDC",
    "REMIND - SSP5-NPI",
    "REMIND - SSP5-PKBUDG1150",
    "REMIND - SSP5-PKBUDG500"
]


In [ ]:
def load_table(scenario, region):
    if scenario.startswith('REMIND'):
        years_scen = [x for x in years if x != 2055]
        techs = techs_remind
        regions = regions_remind
    elif scenario.startswith('IMAGE'):
        techs = techs_image
        regions = regions_image
    else:
        print('No such scenario exists.')
        return
    row_pos = regions.index(region)
    if scenario.startswith('REMIND'):
        start_row = 6 + row_pos * (len(years) + 6)
        end_row = start_row + len(years)
    elif scenario.startswith('IMAGE'):
        start_row = 6 + row_pos * (len(years) + 5)
        end_row = start_row + len(years) -1
    
    col_pos = scenarios.index(scenario)
    start_column = 1 + col_pos * (len(techs) + 4)
    if scenario.startswith('REMIND'):
        start_column = 1 + 4 * (len(techs_image) + 4) + (col_pos - 4) * (len(techs) + 4)
    end_column = start_column + len(techs)
    columns_to_load = xlsxwriter.utility.xl_col_to_name(start_column)+':'+xlsxwriter.utility.xl_col_to_name(end_column)
    
    df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl', skiprows=range(1, start_row + 1), nrows=end_row - start_row + 1, usecols=columns_to_load)
    df.columns = df.iloc[0]
    df = df[1:]
    df = df.set_index([df.columns[0]])
    
    return df

In [ ]:
data = {}
for region in regions_remind: 
    data[region] = load_table(scenario, region)

In [ ]:
# Assuming you have a list of dataframes called list_of_dataframes
result_df = data['CAZ'].copy()
result_df[:] = 0
for reg in data:
    result_df += data[reg]
df = result_df

### Prepare data for pie chart

In [ ]:
# Combine and rename columns
df = df.assign(
    Biomass=df['Biomass CHP'] + df['Biomass IGCC'],
    Solar=df['Solar CSP'] + df['Solar PV Centralized'],
    Wind=df['Wind Onshore'] + df['Wind Offshore'],
    Coal_CCS=df['Coal PC CCS'] + df['Coal IGCC CCS'],
    Coal=df['Coal PC'] + df['Coal IGCC'] + df['Coal CHP'],
    Gas=df['Gas OC'] + df['Gas CC'] + df['Gas CHP']
)

In [ ]:
# Rename specific columns
column_rename_dict = {
    'Biomass IGCC CCS': 'Biomass CCS',
    'Gas CC CCS': 'Gas CCS',
    'Oil ST': 'Oil',
    'Coal_CCS': 'Coal CCS'}

df = df.rename(columns=column_rename_dict)

In [ ]:
# Drop the original columns that were combined
df = df.drop(columns=['Biomass CHP', 'Biomass IGCC', 'Wind Onshore', 'Wind Offshore',
                      'Solar CSP', 'Solar PV Centralized',
                      'Coal PC CCS', 'Coal IGCC CCS',
                      'Coal PC', 'Coal IGCC', 'Coal CHP',
                      'Gas OC', 'Gas CC', 'Gas CHP'])

In [ ]:
# Desired order
reorder = ["Biomass CCS", "Biomass", "Solar", "Wind", "Geothermal", "Hydro", "Nuclear", "Coal CCS", "Gas CCS", "Coal", "Gas", "Oil"]
df = df[reorder]

In [ ]:
file_path = 'Input/meoh_results_full_abatement_Base.xlsx'  # Replace with the path to your Excel file
sheet_name = 'choices'

In [ ]:
df_cc = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
# Drop rows that don't contain retrofitt information
df_cc = df_cc.iloc[2:4]
# Drop columns with capacity limit
df_cc = df_cc.drop(df_cc.columns[df_cc.eq(100000000000000000000).all()], axis=1)
# Drop the first column
df_cc = df_cc.iloc[:, 1:]
# Rename the rows
df_cc.index = ['CC', 'no CC']
# Rename columns
df_cc.columns = range(len(df_cc.columns))

In [ ]:
coal, coal_cc, gas, gas_cc = (0, 0, 0, 0)
for i in range(len(df_cc.columns)):
    if i%2 == 0:
        if 'coal' in df_cc[i]['CC'] or 'lignite' in df_cc[i]['CC']:
            coal_cc += df_cc[i+1]['CC']
            coal += df_cc[i+1]['no CC']
        elif 'gas' in df_cc[i]['CC']:
            gas_cc += df_cc[i+1]['CC']
            gas += df_cc[i+1]['no CC']
retrofitt = {'coal cc': coal_cc*3.6*1e-12,
             'coal': coal*3.6*1e-12,
             'gas cc': gas_cc*3.6*1e-12,
             'gas': gas*3.6*1e-12}
coal_share = retrofitt['coal cc'] / (retrofitt['coal cc'] + retrofitt['coal'])
gas_share = retrofitt['gas cc'] / (retrofitt['gas cc'] + retrofitt['gas'])

In [ ]:
# Select the row you want to plot as a pie chart
row_to_plot = df.loc[year] 
row_to_plot_copy = copy.deepcopy(row_to_plot)

In [ ]:
del row_to_plot_copy['Gas']
del row_to_plot_copy['Oil']
row_to_plot_copy['Coal'] = row_to_plot['Coal'] * (1-gas_share)
row_to_plot_copy['Coal CCU'] =  row_to_plot['Coal'] * gas_share
row_to_plot_copy['Gas'] = row_to_plot['Gas'] * (1-gas_share) 
row_to_plot_copy['Gas CCU'] =  row_to_plot['Gas'] * gas_share
row_to_plot_copy['Oil'] = row_to_plot['Oil']

### Plotting 🎨

In [ ]:
import matplotlib.pyplot as plt

# Select the row you want to plot as a pie chart
row_to_plot = df.loc[year]  # You can replace 2005.0 with the index of the row you want to plot

# Define a custom color palette
colors = {
    'Biomass CCS': '#6ed503',
    'Biomass': '#9fd4e4',
    'Solar': '#5db6e4',
    'Wind': '#0077ba',
    'Geothermal': '#0056a3',
    'Hydro': '#00337a',
    'Nuclear': '#011262',
    'Coal CCS': 'purple',
    'Gas CCS': 'indigo',
    'Coal': '#777777',
    'Coal CCU': 'red',
    'Gas': '#555555',
    'Gas CCU': 'darkred',
    'Oil': '#333333'
}

# Create a pie chart with custom colors and black outlines
plt.figure(figsize=(8, 8))
patches, texts, autotexts = plt.pie(row_to_plot_copy, labels=None, autopct='',wedgeprops = {'linewidth': 3} , startangle=0, colors=[colors[label] for label in row_to_plot_copy.index])

# Set black outline for each pie slice
for patch in patches:
    patch.set_edgecolor('black')

plt.title('')  # Hide the title of the pie chart
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Create a legend with 4 columns and 3 rows, centered at the top
#legend = plt.legend(labels=row_to_plot_copy.index, loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=5)

# Save the figure with a DPI of 600
plt.savefig('Output/Figure 4 pie - '+ scenario + '_' + str(year) +'.png', dpi=600)

# Show the pie chart
plt.show()

In [ ]:
pd.DataFrame(row_to_plot_copy).to_excel('Output/Figure 4 pie - '+ scenario + '_' + str(year) +'.xlsx')